In [1]:
import dask
from dask.distributed import Client
import dask.array as da
import numpy as np
import time
from pathlib import Path
import dask.dataframe as dd
from rdkit import Chem
from mol2vec.features import mol2alt_sentence
from gensim.models import Word2Vec
import dask.bag as db

Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


In [2]:
shrunken_directory = Path("../data/shrunken")

In [3]:
dtypes = {'buildingblock1_smiles': np.int16, 'buildingblock2_smiles': np.int16, 'buildingblock3_smiles': np.int16,
          'binds_BRD4':np.byte, 'binds_HSA':np.byte, 'binds_sEH':np.byte}

In [4]:
client = Client()

In [5]:
import pandas as pd
train_df = pd.read_parquet(shrunken_directory / "train.parquet")

In [18]:
molecule_smiles = da.from_array(train_df.molecule_smiles.to_list(), chunks=(1000000,)) 

In [19]:
# Apply mol2vec to the molecule_smiles array
def mol2vec(smiles):
    sentence = mol2alt_sentence(Chem.MolFromSmiles(smiles), 1)
    return sentence

molecule_sentences = molecule_smiles.map_blocks(mol2vec, dtype='object')

In [20]:
molecule_sentences

dask.array<mol2vec, shape=(98415610,), dtype=object, chunksize=(1000000,), chunktype=numpy.ndarray>

In [ ]:
molecule_sentences.compute()

dask.array<mol2vec, shape=(98415610,), dtype=object, chunksize=(1000000,), chunktype=numpy.ndarray>